In [ ]:
""" Install eppy library if needed """
#pip install eppy

In [ ]:
""" 
Initially there was always error, such as:
        make_idd_index() missing 1 required positional argument: 'debug'
updating of this library helped to solve the issue 
    
"""
pip install -U decorator    

In [15]:
from eppy import modeleditor
from eppy.modeleditor import IDF
# iddfile="C:/EnergyPlusV8-9-0/Energy+.idd" here must be that to *.idd file of installed EnergyPlus on local machine
iddfile = "../data/Energy+.idd" 
fname1 = "../data/DeustoTest.idf"
IDF.setiddname(iddfile)
idf1 = IDF(fname1)

To access specific object of idf file we basically create the object in python *building* which is the instant of the particular element in the idf file, such as in our case idf1.idfobjects['Building'][0], and this newely created object ve can access all the variables (fields) as it would be the methods of the class.


In [17]:
building = idf1.idfobjects['Building'][0]

building.Name = "Testig building change"
# idf1.printidf() #print the whole idf file
print(idf1.idfobjects['Building'])

[
Building,
    Testig building change,    !- Name
    0,                        !- North Axis
    Suburbs,                  !- Terrain
    0.04,                     !- Loads Convergence Tolerance Value
    0.4,                      !- Temperature Convergence Tolerance Value
    FullExterior,             !- Solar Distribution
    25,                       !- Maximum Number of Warmup Days
    6;                        !- Minimum Number of Warmup Days
]


**Looping through E+ objects**

The export of parameters will be made with the pandas dataframe. After that the each component would be considered as list

In [63]:
import pandas as pd
df= pd.read_excel('../data/matrix.xlsx')    
df.head()

,ESCENARIO,Façade,Huecos (Uglass),Windows (SHGC),Windows (Transmisión de luz),Infiltration
0,"0,0,0,0",0.000,3.41,0.781,0.801,0.61
1,1.0.0.0.,0.042,3.42,0.782,0.802,0.62
2,1.1.1.0.,0.043,3.43,0.693,0.803,0.63


at first we specify exact components that we want to access and change the values

Check the length of the elements in the newely created wariable to know which i-th object's variable (i.e. correspondent parameter of the i-th element in a list) must be changed

In [64]:
#len (materials)
len (construction)
facade_material = idf1.idfobjects["MATERIAL"][11] # we want tj change EPS Expanded Polystyrene thickness in Facade
construction_material = idf1.idfobjects["Construction"][6]#not clear!
window_material = idf1.idfobjects['WindowMaterial:SimpleGlazingSystem'][0] # ti change Uvalue, SHGC, Transmisión de luz)

Check the selected element

In [58]:
print (facade_material)
print (window_material)


Material,
    EPS Expanded Polystyrene (Standard)_O1234,    !- Name
    Rough,                    !- Roughness
    0.04,                     !- Thickness
    0.04,                     !- Conductivity
    15,                       !- Density
    1400,                     !- Specific Heat
    0.9,                      !- Thermal Absorptance
    0.6,                      !- Solar Absorptance
    0.6;                      !- Visible Absorptance

[
WindowMaterial:SimpleGlazingSystem,
    Simple 1001,              !- Name
    1.96,                     !- UFactor
    0.691,                    !- Solar Heat Gain Coefficient
    0.744;                    !- Visible Transmittance
]


Iteration

In [65]:
for i in df['Façade']:
    facade_material.Thickness=i
    print (facade_material)
#[facade_material.Thickness=i for i in df['Façade']]


Material,
    EPS Expanded Polystyrene (Standard)_O1234,    !- Name
    Rough,                    !- Roughness
    0,                        !- Thickness
    0.04,                     !- Conductivity
    15,                       !- Density
    1400,                     !- Specific Heat
    0.9,                      !- Thermal Absorptance
    0.6,                      !- Solar Absorptance
    0.6;                      !- Visible Absorptance


Material,
    EPS Expanded Polystyrene (Standard)_O1234,    !- Name
    Rough,                    !- Roughness
    0.042,                    !- Thickness
    0.04,                     !- Conductivity
    15,                       !- Density
    1400,                     !- Specific Heat
    0.9,                      !- Thermal Absorptance
    0.6,                      !- Solar Absorptance
    0.6;                      !- Visible Absorptance


Material,
    EPS Expanded Polystyrene (Standard)_O1234,    !- Name
    Rough,                    !- Roug

How to iterate through an oblect and change necesarry parameters in one iteration (i.e. correspondednt raw of the Dataframe table inserted into the proper name of the parameter)

In [67]:
for index, row in df.iterrows():
    window_material.UFactor= row['Huecos (Uglass)']
    window_material.Solar_Heat_Gain_Coefficient = row['Windows (SHGC)']
    window_material.Visible_Transmittance = row['Windows (Transmisión de luz)']
    print (window_material)


WindowMaterial:SimpleGlazingSystem,
    Simple 1001,              !- Name
    3.41,                     !- UFactor
    0.781,                    !- Solar Heat Gain Coefficient
    0.801;                    !- Visible Transmittance


WindowMaterial:SimpleGlazingSystem,
    Simple 1001,              !- Name
    3.42,                     !- UFactor
    0.782,                    !- Solar Heat Gain Coefficient
    0.802;                    !- Visible Transmittance


WindowMaterial:SimpleGlazingSystem,
    Simple 1001,              !- Name
    3.43,                     !- UFactor
    0.693,                    !- Solar Heat Gain Coefficient
    0.803;                    !- Visible Transmittance



**Mid term conclusion: Optimization of looping must be DONE!!!**

TO BE CONTINUED ...